# Running Rank 1 models

In this notebook, we will run rank 1 model with another dataset. At 2022 competition, rank 1 model excelled in Multiome prediction task, so we'll focus on it. CITE-seq model leverages a lot of the same code and has a very similar architecture, so it should be possible to run it as well, just put CITE-seq data instead of Multiome in this tutorial and change task_type to "cite" in the scripts.

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from scanpy.preprocessing._utils import _get_mean_var
import muon

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download 2021 competition data from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE194122

In [2]:
adata = sc.read_h5ad("../data/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad")

The dataset has both modalities, ATAC-seq and RNA in one AnnData object. Let's split modalities

In [3]:
adata

AnnData object with n_obs × n_vars = 69249 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [4]:
adata.var["feature_types"].value_counts()

feature_types
ATAC    116490
GEX      13431
Name: count, dtype: int64

In this example notebook, we'll subset data to a small number of cells for faster testing

In [5]:
random_cells = np.random.choice(adata.n_obs, size=500, replace=False)

In [6]:
adata = adata[random_cells, :].copy()

In [7]:
adata_rna = adata[:, adata.var["feature_types"] == "GEX"]
adata_rna

View of AnnData object with n_obs × n_vars = 500 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [8]:
adata_atac = adata[:, adata.var["feature_types"] == "ATAC"]
adata_atac

View of AnnData object with n_obs × n_vars = 500 × 116490
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

Let's further reduce the number of features to 500 for RNA and 1000 for ATAC for faster testing.

In [9]:
adata_rna = adata_rna[:, np.random.choice(adata_rna.var_names, size=500, replace=False)]
adata_rna

View of AnnData object with n_obs × n_vars = 500 × 500
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [10]:
adata_atac = adata_atac[:, np.random.choice(adata_atac.var_names, size=1000, replace=False)]
adata_atac

View of AnnData object with n_obs × n_vars = 500 × 1000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

Before modality prediction, apply QC to your dataset, otherwise the code will likely fail. Here, we'll simply remove cells with 0 RNA counts or open chromatin regions, and filter out constant features.

In [11]:
non_empty_cells = (adata_rna.X.sum(axis=1) != 0) & (adata_atac.X.sum(axis=1) != 0)

In [12]:
adata_rna = adata_rna[non_empty_cells, :]
adata_atac = adata_atac[non_empty_cells, :]

In [13]:
rna_means, rna_vars = _get_mean_var(adata_rna.X)
adata_rna = adata_rna[:, rna_vars != 0]
adata_rna.shape

(500, 497)

In [14]:
atac_means, atac_vars = _get_mean_var(adata_atac.X)
adata_atac = adata_atac[:, atac_vars != 0]
adata_atac.shape

(500, 969)

Normalize ATAC data with TF-IDF. Note that RNA data must be library-size and log1p normalized as well.

In [15]:
muon.atac.pp.tfidf(adata_atac)
adata_atac.X[:10, :10].toarray()

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/muon/_atac/preproc.py:84: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


array([[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       , 10.769051 ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  7.7392497],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  8.866014 ],
       [ 0.       ,  0.       ,  0.      

In [16]:
adata_rna.write_h5ad("../data/adata_rna_subset.h5ad")
adata_atac.write_h5ad("../data/adata_atac_subset.h5ad")

# Format the data correctly

To run the model, we need to save our data to the following files:
- test_multi_inputs.h5
- train_multi_inputs.h5
- train_multi_targets.h5

If you want to use CITE-seq model too, the you'll additionally need:
- test_cite_inputs
- train_cite_inputs
- train_cite_targets

Additionally, we must save test set labels to the "evaluation_ids.csv" file with "cell_id" column

In [17]:
adata_rna.obs["DonorID"].value_counts()

DonorID
15078    126
19593     85
18303     61
28483     52
12710     42
10886     40
16710     39
13272     27
28045     18
11466     10
Name: count, dtype: int64

Let's use donor 15078 as a test

In [18]:
adata_rna.obs["split"] = "train"
adata_rna.obs.loc[adata_rna.obs["DonorID"] == 15078, "split"] = "test"
adata_atac.obs["split"] = adata_rna.obs["split"]

/var/folders/p9/74j47tdn36n6s2vcv_5079q0s4wk4h/T/ipykernel_38261/2536117573.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_rna.obs["split"] = "train"


In [19]:
adata_rna[adata_rna.obs["split"] == "train"].write_h5ad("../data/train_multi_targets.h5")
adata_atac[adata_atac.obs["split"] == "train"].write_h5ad("../data/train_multi_inputs.h5")
adata_atac[adata_atac.obs["split"] == "test"].write_h5ad("../data/test_multi_inputs.h5")

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


We also need to save metadata to a file. Note that in the current implementation, the metadata column **must** be named "day", "donor", "technology", "cell_type", and "cell_id". Day must be a number

In [20]:
adata_rna.obs.columns

Index(['GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors',
       'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments',
       'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction',
       'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order',
       'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality',
       'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType',
       'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker',
       'split'],
      dtype='object')

In [21]:
adata_rna.obs["day"] = adata_rna.obs["batch"].str[3:]
adata_rna.obs["donor"] = adata_rna.obs["DonorID"]
adata_rna.obs["technology"] = adata_rna.obs["Modality"]
adata.obs.index.name = "cell_id"

In [22]:
adata_rna.obs[["day", "donor", "technology", "cell_type"]].reset_index(names="cell_id").to_csv("../data/metadata.csv")

Small function to save the data in the appropriate format:

In [23]:
def save_X_to_h5(adata, path):
    from pathlib import Path
    path = Path(path)
    X = adata.X.A if hasattr(adata.X, "A") else adata.X  # make dense if sparse
    df = pd.DataFrame(X, index=adata.obs_names, columns=adata.var_names)
    df.to_hdf(
        path,
        key=path.name,  # e.g. "train_multi_targets"
        mode="w",
        format="fixed",
    )

save_X_to_h5(adata_rna[adata_rna.obs["split"] == "train"], "../data/train_multi_targets.h5")
save_X_to_h5(adata_atac[adata_atac.obs["split"] == "train"], "../data/train_multi_inputs.h5")
save_X_to_h5(adata_atac[adata_atac.obs["split"] == "test"], "../data/test_multi_inputs.h5")

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'train_multi_targets.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'train_multi_inputs.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 't

Run scripts as recommended by competitor. For the given subset of data, it takes ~30 minutes on Macbook (cpu mode). If you wish to use CITE-seq model, change `task_type` to `cite` in the last script. You might also want to download additional files with biological priors for CITE-seq prediction. See [readme](https://github.com/lueckenlab/OpenProblems2022Analysis/tree/main/code/rank1/open-problems-multimodal) of the original repository for details

In [24]:
%%bash

export DATA_DIR=/Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/

cd rank1/open-problems-multimodal/
python3 script/make_compressed_dataset.py --data_dir ${DATA_DIR}
python3 script/make_additional_files.py --data_dir ${DATA_DIR}
python3 script/make_compressed_dataset.py --data_dir ${DATA_DIR}
python3 script/train_model.py --data_dir ${DATA_DIR} --task_type multi 

File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/evaluation_ids.csv does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/sample_submission.csv does not exist
126
374
374
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/test_cite_inputs.h5 does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/train_cite_inputs.h5 does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/train_cite_targets.h5 does not exist
Some citeseq files don't exist, not making citeseq cell statistics
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/evaluation_ids.csv does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/sample_submission.csv does not exist
126
374
374
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/test_cite_inputs.h5

The results are saved as pickled numpy array. Let's read them and calculate a correlation score to ground truth. We can import a function to compute the metric directly from competitor's code:

In [25]:
import sys
sys.path.append("rank1/open-problems-multimodal")  # Make it possible to import from competitor's code

from ss_opm.metric.correlation_score import correlation_score

In [26]:
import pickle

with open("rank1/open-problems-multimodal/result/multimodal_pred.pickle", "rb") as f:
    y_pred = pickle.load(f)

In [27]:
y_pred.shape

(126, 497)

In [28]:
y_pred

array([[ 0.0813081 ,  0.0371041 , -0.25203303, ..., -0.27175885,
        -0.14356263, -0.31657338],
       [-0.6202765 ,  0.2970362 , -0.2643033 , ..., -0.32891843,
        -0.00652082, -0.37576196],
       [ 3.0143929 , -0.16087617, -0.48203787, ..., -0.3071232 ,
        -0.3912091 , -0.4892507 ],
       ...,
       [ 1.8792413 ,  0.41418588, -0.45789668, ..., -0.33398178,
        -0.05135709, -0.388742  ],
       [ 3.2200477 , -0.09397589, -0.48668027, ..., -0.3343542 ,
        -0.3164395 , -0.4438851 ],
       [ 1.1289066 , -0.38751778, -0.29504016, ..., -0.03988038,
        -0.22412674, -0.2041503 ]], dtype=float32)

In [29]:
y_true = adata_rna[adata_rna.obs["split"] == "test"].X
correlation_score(y_true, y_pred)

0.3395417297927301